In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy  as np
import pandas as pd

import os

DETDIR='results/Main/google'
#ANNDIR='Annotations'
#TRIDIR='ImageSets/Main/'

In [ ]:
detections = os.listdir(DETDIR)

In [ ]:
!ls $DETDIR

In [ ]:
frames = []
for det in detections:
    frames.append(pd.read_csv(os.path.join(DETDIR,det),
                    delimiter=' ',names=['file','prob','xmin','ymin','xmax','ymax','pred_class']) )

In [ ]:
pred_df = pd.concat(frames,ignore_index=True)

In [ ]:
#Check a single result
pred_df[pred_df.file == 'muminus008457'].sort('prob',ascending=False).iloc[0:5]

In [ ]:
pred_df.file.unique()

In [ ]:
import re
truth = re.compile("^([a-z]+)[0-9]+")
pars = ['','eminus','proton','pizero','muminus','gamma']

def get_gt(row): 
    return pars.index(truth.search(row.file).group(1))
    

In [ ]:
# Go row by row and mark the groundtruth
pred_df['gt'] = pred_df.apply(get_gt,axis=1)

In [ ]:
ious = []
probs = []
im = {}
for ix,image in enumerate(pred_df.file.unique()):
    df = pred_df[pred_df['file'] == image].sort('prob',ascending=False)
    top = df.iloc[0] # take only the top prediction
    
    im[image] = {'gt':top['gt'],'top_pred_class':top['pred_class'],'top_prob':top['prob']}

In [ ]:
im_df = pd.DataFrame(im).T

In [ ]:
ar = lambda x, y : np.arange(0,x,y)

In [ ]:
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['font.family']='serif'

In [ ]:
inc = 0.01

for pred in xrange(5):
    pred += 1
    a = {}
    for i in xrange(5):
        i += 1    
        res = []
        for j in ar(1,inc):
            bot = im_df[im_df['top_pred_class'] == pred].query('top_prob > {}'.format(j))
            top = bot.query('gt == {}'.format(i))
            bot = float(bot.values.size)
            top = float(top.values.size)
            r = top/bot if bot > 0.0 else 0.0
            res.append(r * 100.0)
        a[i] = res
    fig,ax = plt.subplots(figsize=(10,6))
    ax.hist([ar(1,inc) for _ in range(5)],bins=ar(1+inc,inc),
             weights=[a[r] for r in a],
             stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
    ax.set_ylim(0,100)
    ax.legend(loc='best',fontsize=12)
    ax.set_title("FRCNN Detection: {}".format(pars[pred]))
    ax.set_ylabel("Percent Truth")
    ax.set_xlabel("Prob. Threshold")
    plt.savefig('detection_truth_{}.pdf'.format(pars[pred]), format='pdf', dpi=1000)
    plt.show()

In [ ]:
inc = 0.01

for pred in xrange(5):
    pred += 1
    a = {}
    b = {}
    for i in xrange(5):
        i += 1    
        res = []
        ress = []
        for j in ar(1,inc):
            bot = im_df[im_df['gt'] == pred].query('top_prob > {}'.format(j))
            top = bot.query('top_pred_class == {}'.format(i))
            bot = float(bot.values.size)
            top = float(top.values.size)
            r = top/bot if bot > 0.0 else 0.0
            res.append(r * 100.0)
            ress.append(bot)
        a[i] = res


    fig,ax = plt.subplots(figsize=(10,6))
    ax.hist([ar(1,inc) for _ in range(5)],bins=ar(1+inc,inc),
             weights=[a[r] for r in a],stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
    ax.set_ylim(0,100)
    ax.legend(loc='best',fontsize=12)
    ax.set_title("Truth: {}".format(pars[pred]))
    ax.set_ylabel("Percent Detected")
    ax.set_xlabel("Prob. Threshold")
    plt.show()